In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyIEEM.data.utils import to_pd_interval

# Regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm

## Settings

In [2]:
df = 3
degree = 2
data_df = pd.read_csv('comesf_SPC.csv', index_col = [0,1,2,3])
age_classes = ['[0, 10)','[10, 20)','[20, 30)','[30, 40)','[40, 50)','[50, 60)','[60, 70)','[70, 80)','[80, 120)']
age_classes_intervalindex = pd.IntervalIndex.from_tuples([(0,10),(10,20),(20,30),(30,40),(40,50),(50,60),(60,70),(70,80),(80,120)], closed='left')

## Functions

In [3]:
import statsmodels.api as sm
from statsmodels.gam.api import GLMGam, BSplines

def smooth_matrix(matrix, age_classes, age_classes_intervalindex, df, degree):
    """
    A function to GAM smooth the matrices
    
    Input
    -----
    
    matrix: pd.Series
        Index: age_x, age_y. Name: predicted_contacts. Unsmoothed matrix.
        
    age_classes: list containing str
        String representation of the age groups
        
    age_classes_intervalindex: pd.Intervalindex
        Corresponding intervalindex
        
    df: int
        Number of B-splines for GAM fit
        
    degree: int
        Degree of B-splines for GAM fit    
    
    Output
    ------
    
    matrix: pd.Series
        Index: age_x, age_y. Name: predicted_contacts. Smoothed matrix.
    """
    
    # drop index
    matrix = matrix.reset_index()

    # replace categoral with midpoint of interval (in case data are not equidistant)
    for var in ['age_x', 'age_y']:
        for i,age_class in zip(range(len(age_classes)), age_classes):
            matrix.loc[matrix.index[matrix[var] == age_class], var] = age_classes_intervalindex[i].mid  
    
    matrix = matrix.astype(float)

    # save the midpoints
    midpoints_x = matrix['age_x'].unique() 
    midpoints_y = matrix['age_y'].unique() 

    # drop nan values before first GAM fit
    matrix = matrix.dropna()
    
    # create spline basis and GAM model
    x_spline = matrix[['age_x', 'age_y']]
    bs = BSplines(x_spline, df=[df,df], degree=[degree, degree])
    model = GLMGam.from_formula('predicted_contacts ~ age_x + age_y + age_x*age_y', data=matrix, smoother=bs,
                                family=sm.families.NegativeBinomial(), alpha=np.array([0.1, 0.1]), method='newton')

    # fit GAM model
    res = model.fit()
    
    # predict matrix of contacts
    smoothed_values = res.predict()
    
    # merge back into dataframe
    matrix['predicted_contacts'] = smoothed_values
    
    # define dataframe with zeros
    names = ['age_x', 'age_y']
    iterables = [midpoints_x, midpoints_y]
    desired_df = pd.Series(0, index=pd.MultiIndex.from_product(iterables, names=names), name='desired_format')
    
    # merge smoothed data into desired format
    mat = matrix.groupby(by=['age_x','age_y']).last()
    mat = mat['predicted_contacts'].combine_first(desired_df)
    
    # unstack dataframe and interpolate the nan values linearily
    mat = mat.unstack(level=1).interpolate(method='linear', limit_area='inside')
    mat = mat.fillna(0)
    mat = mat.stack()
    mat = mat.rename('predicted_contacts')
    
    # recycle name matrix
    matrix = mat.reset_index()
    
    # now fit another GAM including the zeros
    x_spline = matrix[['age_x', 'age_y']]
    bs = BSplines(x_spline, df=[df,df], degree=[degree, degree])
    model = GLMGam.from_formula('predicted_contacts ~ age_x + age_y + age_x*age_y', data=matrix, smoother=bs,
                                family=sm.families.NegativeBinomial(), alpha=np.array([0.1, 0.1]), method='newton')

    # fit GAM model
    res = model.fit()
    
    # predict matrix of contacts
    smoothed_values = res.predict()
    
    # merge back into dataframe
    matrix['predicted_contacts'] = smoothed_values
    
    # set multiindex
    matrix = matrix.groupby(by=['age_x','age_y']).last()

    # re-introduce the age classes as index
    matrix.index = matrix.index.set_levels(age_classes, level=0)
    matrix.index = matrix.index.set_levels(age_classes, level=1)

    return matrix.squeeze()

In [4]:
def construct_matrix(data, age_classes, age_classes_intervalindex, df, degree):
    """
    A function to perform GEE on the raw contact data and smoothing of the resulting matrix with GAM
    
    Input
    -----
    
    data: pd.DataFrame
       Index: ID, age_x, age_y (pd.Multiindex). Values: reported number of contacts.
    
    age_classes: list containing str
        String representation of the age groups
        
    age_classes_intervalindex: pd.Intervalindex
        Corresponding intervalindex
        
    df: int
        Number of B-splines for GAM fit
        
    degree: int
        Degree of B-splines for GAM fit
    
    Output
    ------
    
    data_GEE: pd.Dataframe
        Index: age_x, age_y. Contact matrix after GEE. Contains nan where no observation exists.
        
    data_smooth: pd.DataFrame
        Index: age_x, age_y. Contact matrix after GEE and GAM smoothing.
        
    n: int
        Number of participants
    """
    
    # extract number of individuals present in dataset
    n = len(data.index.get_level_values('ID').unique())
    
    # use name 'reported_contacts' for input data
    data = data.rename('reported_contacts')

    # reset index on data
    data = data.reset_index()
    
    # GEE regression
    fam = sm.families.NegativeBinomial()
    ind = sm.cov_struct.Independence()
    mod = smf.gee("reported_contacts ~ age_y + age_x", "ID", data, cov_struct=ind, family=fam)
    res = mod.fit()
    
    # format data
    data['predicted_contacts'] = res.predict()
    data = data.drop(columns=['ID'])
    data = data.groupby(by=['age_x', 'age_y']).last()
    
    # merge with a pre-made dataframe containing all entries (nan where no prediction exists)
    names = ['age_x', 'age_y']
    iterables = 2*[age_classes,]
    desired_df = pd.Series(index=pd.MultiIndex.from_product(iterables, names=names), name='desired_format', dtype=float)
    
    # data post GEE
    data_GEE = data['predicted_contacts'].combine_first(desired_df)

    # smooth using GAM
    data_smooth = smooth_matrix(data['predicted_contacts'].combine_first(desired_df),
                                    age_classes, age_classes_intervalindex, df, degree)
    
    return data_GEE, data_smooth, n

## Analysis

In [5]:
for sector in data_df.index.get_level_values('sector').unique().values:

    # GEE and GAM smooth
    data_GEE, data_smooth, n = construct_matrix(data_df.loc[slice(None), slice(None), sector, slice(None)].squeeze(),
                                             age_classes, age_classes_intervalindex, df, degree)
    
    # Extract matrix
    mat = data_GEE.values.reshape(2*[len(age_classes),])
    mat_smooth = data_smooth.values.reshape(2*[len(age_classes),])

    # Visualise matrix smoothed vs. unsmoothed
    titles=['unsmoothed', f'GAM smoothed ({df}, {degree})']
    fig,axs=plt.subplots(nrows=1, ncols=2, sharey=True)
    for i,m in enumerate([mat, mat_smooth]):
        ax = axs[i]
        ax = sns.heatmap(m, annot=True, fmt='.1f', ax=ax, square=True, cbar=False, annot_kws={"size":7})
        ax.xaxis.tick_top() # x axis on top
        ax.xaxis.set_label_position('top')
        ax.set_xticklabels(age_classes, rotation = 30, size=8)
        ax.set_yticklabels(age_classes, rotation = 30, size=8)
        ax.set_title(f'{titles[i]}\navg. contacts: {np.mean(np.sum(m,axis=1)):.1f}', fontsize=10)

        if i == 0:
            ax.set(ylabel='participant age')

    fig.suptitle(f'Sector {sector} (n={n})')
    plt.tight_layout()
    plt.savefig(f'matrices/mat_{sector}.png')
    plt.close()